In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# %cd "/content/drive/MyDrive/data"

In [ ]:
# !wget https://www.image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar #-p /content/drive/MyDrive/data

In [ ]:
# !wget https://www.image-net.org/data/ILSVRC/2012/ILSVRC2012_img_train.tar #-p /content/drive/MyDrive/data

In [ ]:
# %ls

In [ ]:
# %cd "/content/drive/MyDrive/data"

In [ ]:
# # !chmod +x extract_ILSVR.sh

# !sh /content/drive/MyDrive/data/extract_ILSVRC.sh

## ImageNet setting

In [ ]:
import random

import torchvision
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader, Subset

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Function
from torch.cuda.amp import autocast, GradScaler

from torch.nn.modules.batchnorm import BatchNorm2d

from matplotlib import pyplot as plt
import numpy as np
import math
import copy
import time
import os
import json


from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/MyDrive/Colab Notebooks"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks


checkpoint vs. from scratch 확인

In [ ]:
def set_random_seeds(random_seed=0):

  torch.manual_seed(random_seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False
  np.random.seed(random_seed)
  random.seed(random_seed)

set_random_seeds(10)

In [ ]:
# # !python -m torch.distributed.launch --nproc_per_node 1 ImageNet/DDP.py
# !torchrun --nnodes 1 --nproc_per_node 2 DDP.py

In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks"
!python DDP.py --my_model "ResNet18_WCPC_1000" --lr 1e-2 --w_mode 'Array' --ps_mode 'Array' --w_ch --ps_ch #--warmstart

/content/drive/MyDrive/Colab Notebooks
FINETUNE [ ResNet18_WCPC_1000 ]
BEST EVAL LOADED: 42.899999022483826 %
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "
DATALOADER...
----------------------------------------
                Training                
----------------------------------------
	# corrects: 2 1 2 0 1 0 0 1 1 0 1 1 0 0 0 1 0 0 1 1 0 1 2 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 1 0 0 0 0 1 1 2 0 0 0 0 1 0 3 0 0 0 2 0 0 2 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 
Epoch: 000 Eval Loss: 16.8656 Eval Acc: 0.0008
	# corrects: 2 1 0 0 0 2 0 1 0 1 1 1 1 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 1 1 0 2 1 0 0 0 0 0 1 3 0 0 0 0 0 1 1 0 1 1 1 1 0 0 0 0 0 0 6 0 1 0 1 1 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 2 1 0 0 0 0 1 1 1 0 0 0 1 1 1 0 0 2 1 1 2 0 0 3 0 1 0 0 0 

In [ ]:
print(torch.cuda.device_count())

#### w/ grad accumulation + mixed precision training

In [ ]:
''' Gradient accumulation '''

arch_tag2 = "WLPA_finetune"

def retrain_model(model, optimizer, train_loader, test_loader, device, learning_rate=5e-2, epochs=90):
    criterion = nn.CrossEntropyLoss()

    model.to(device)
    scaler = GradScaler()

    # optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=3e-3)
    # #scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[60, 120, 160], gamma=0.1, last_epoch=-1)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30, eta_min=0, last_epoch= -1, verbose=False) # T_max ...


    # # Define optimizer and scheduler for stage 1
    # optimizer_stage1 = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-3)
    # scheduler_stage1 = torch.optim.lr_scheduler.CosineAnnealingLR(
    #     optimizer_stage1, T_max=40, eta_min=1e-4, last_epoch=-1, verbose=False)

    # # Define optimizer and scheduler for stage 2
    # optimizer_stage2 = optim.SGD(model.parameters(), lr=learning_rate/5, momentum=0.9, weight_decay=5e-3)
    # scheduler_stage2 = torch.optim.lr_scheduler.CosineAnnealingLR(
    #     optimizer_stage2, T_max=60, eta_min=1e-4, last_epoch=-1, verbose=False)


    best_eval = -1
    best_epoch = 0

    model.eval()
    eval_loss, eval_accuracy = evaluate_model(model=model, test_loader=test_loader,
                                            device=device, criterion=criterion)
    print("Epoch: {:03d} Eval Loss: {:.4f} Eval Acc: {:.4f}".format(0, eval_loss, eval_accuracy))

    trainLossTensor = torch.empty((epochs,), dtype=torch.float64)
    evalLossTensor = torch.empty((epochs,), dtype=torch.float64)
    epochTensor = torch.empty((epochs,), dtype=torch.int64)
    evalAccTensor = torch.empty((epochs,), dtype=torch.float64)

    accumulation_steps = 4

    %cd "/content/drive/MyDrive/Colab Notebooks/Psum quant/ImageNet"
    for epoch in range(epochs):
    #   if epoch >= 200:
    #     optimizer = optimizer_stage2
    #     scheduler = scheduler_stage2
    #     for i in range(9):
    #         model.layers[i].conv0.psumOpt = True
    #         model.layers[i].conv1.psumOpt = True
    #   else:
    #     optimizer = optimizer_stage1
    #     scheduler = scheduler_stage1
    #     for i in range(9):
    #         model.layers[i].conv0.psumOpt = False
    #         model.layers[i].conv1.psumOpt = False

        # optimizer = optimizer_stage1
        # scheduler = scheduler_stage1

        # Training
        epochTensor[epoch] = epoch
        model.train()
        running_loss = 0
        running_corrects = 0

        for i, (inputs, labels) in enumerate(train_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            with autocast():
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels) / accumulation_steps
            scaler.scale(loss).backward()

            # optimizer.zero_grad()
            # outputs = model(inputs)
            # _, preds = torch.max(outputs, 1)
            # loss = criterion(outputs, labels) / accumulation_steps
            # loss.backward()

            # # gradient clipping
            # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
            if (i + 1) % accumulation_steps == 0:
                # optimizer.step()  # Update model weights
                # optimizer.zero_grad()  # Reset gradients
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()

            running_loss += loss.item() * inputs.size(0) * accumulation_steps
            running_corrects += torch.sum(preds == labels.data)

        # print('TIME ELAPSED GROUP CONV:', time.perf_counter()-start)
        train_loss = running_loss / len(train_loader.dataset)
        train_accuracy = running_corrects / len(train_loader.dataset)
        trainLossTensor[epoch] = train_loss

        # Evaluation
        model.eval()
        eval_loss, eval_accuracy = evaluate_model(model=model,
                                                test_loader=test_loader,
                                                device=device, criterion=criterion)
        evalLossTensor[epoch] = eval_loss
        evalAccTensor[epoch] = eval_accuracy

        scheduler.step()
        print("Epoch: {:03d} Train Loss: {:.4f} Train Acc: {:.4f} Eval Loss: {:.4f} Eval Acc: {:.4f}".format(epoch+1, train_loss, train_accuracy, eval_loss, eval_accuracy))
        # ####################################
        # if epoch%10==0:
        #     cnt=0
        #     for i in range(9):
        #         ps0 = model.layers[i].conv1.split_conv[0].ps_quan_fn[0]
        #         # ps1 = model.layers[i].conv1.split_conv[1].ps_quan_fn[0]

        #         int0 = ps0.x_q_int
        #         clip0 = ps0.x_q_int_clip
        #         # int1 = ps1.x_q_int
        #         # clip1 = ps1.x_q_int_clip

        #         plt.subplot(3,3,cnt+1)
        #         plt.ticklabel_format(style = 'plain')
        #         plt.hist(int0.cpu().detach().numpy().flatten(), label='x_int0')
        #         plt.hist(clip0.cpu().detach().numpy().flatten(), alpha=0.8, label='x_clip0')
        #         # plt.legend()
        #         cnt += 1
        #     plt.show()
        #     # print(model.layers[0].conv1.split_conv[0].ps_quan_fn[0].sf)
        # ####################################

        # Save best model
        if eval_accuracy > best_eval:
            best_eval = eval_accuracy
            best_epoch = epoch+1

            print('Saving..')
            state = {
                'net': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'acc': best_eval,
                'epoch': epoch,
            }
            if not os.path.isdir('checkpoint'):
                os.mkdir('checkpoint')
            torch.save(state, './checkpoint/'+arch_tag2+'_ckpt.pt')

        if epoch%10 == 0:
            plt.figure(figsize=(5,3))
            plt.grid(color='k', linestyle='--', linewidth=1)
            plt.scatter(epochTensor[:epoch+1], trainLossTensor[:epoch+1], s=20, label='Train Loss')
            plt.scatter(epochTensor[:epoch+1], evalLossTensor[:epoch+1], s=20, label='Eval Loss')

            plt.title('Train Loss vs. Eval Loss')
            plt.xlabel('epochs')
            plt.ylabel('loss')
            plt.legend(loc='upper right')

            plt.show()

        # # Save train log
        # loss_np = evalLossTensor.cpu().numpy()  #convert to Numpy array
        # df = pd.DataFrame(loss_np)              #convert to a dataframe
        # df.to_csv("./loss log/evalLoss_"+arch_tag2+".csv",index=False)       #save to file

        # acc_np = evalAccTensor.cpu().numpy()  #convert to Numpy array
        # df = pd.DataFrame(acc_np)              #convert to a dataframe
        # df.to_csv("./loss log/evalAcc_"+arch_tag2+".csv",index=False)       #save to file

    print("Best eval accuracy: {:.4f} @ epoch {:03d}".format(best_eval, best_epoch))

    # plot
    plt.figure(figsize=(10,6))
    plt.grid(color='k', linestyle='--', linewidth=1)
    plt.scatter(epochTensor, trainLossTensor, s=20, c='m', label='Train Loss')
    plt.scatter(epochTensor, evalLossTensor, s=20, c='c', label='Eval Loss')

    plt.title('Train Loss vs. Eval Loss')
    plt.xlabel('epochs')
    plt.ylabel('loss')
    plt.legend(loc='upper right')

    plt.show()

    return model

In [ ]:
def calibrate_model(model, loader, device=torch.device('cpu')):
  model.to(device)
  model.eval()

  for inputs, labels in loader:
    inputs = inputs.to(device)
    labels = labels.to(device)
    _ = model(inputs)

### random seeds / load cifar-100

In [ ]:
def set_random_seeds(random_seed=0):

  torch.manual_seed(random_seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False
  np.random.seed(random_seed)
  random.seed(random_seed)

# batch_size=256

# transform_train = transforms.Compose([
#     transforms.RandomCrop(32, padding=4),  #resises the image so it can be perfect for our model.
#     transforms.RandomHorizontalFlip(), # FLips the image w.r.t horizontal axis
#     transforms.ToTensor(), # comvert the image to tensor so that it can work with torch
# #   transforms.Normalize((0.507, 0.487, 0.441), (0.267, 0.256, 0.276)) #Normalize all the images
#     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2471, 0.2435, 0.2616))
# ])

# transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
# ])

# trainset = torchvision.datasets.CIFAR100(root='./data',
#                                         train=True,
#                                         download=True,
#                                         transform=transform_train)
# trainloader = torch.utils.data.DataLoader(trainset,
#                                             batch_size=batch_size,
#                                             shuffle=True,
#                                             num_workers=2)
# testset = torchvision.datasets.CIFAR100(root='./data',
#                                         train=False,
#                                         download=True,
#                                         transform=transform)
# testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
#                                         shuffle=False, num_workers=2)



---

